In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split

C:\Users\elkzze1\anaconda\anaconda3\envs\py3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Data

Training data: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

Test data: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

Data dictionary: https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

Task

Create a Random Forest Classifier

Predict whether someone will make greater than $50k per year based on their census data.

Use cross validation

What's the highest accuracy you can get on the test dataset?

United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands

In [46]:
# add a header just because
headers = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','agrossincome']
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', names=headers, index_col=False)
df['over50k'] = df['agrossincome'].apply(lambda x: 1 if x == '>50K' else 0)
# df['country'] = df['native-country'].map( {} )
df['IsMale'] = df['sex'].map({' Female': 0, ' Male': 1})
df['IsMarried'] = df['marital-status'].apply(lambda x: 0 if x == 'Never-married' else 1)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,agrossincome,over50k,IsMale,ISMarried
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,0,1,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,0,1,1
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,0,1,1
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,0,1,1
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,0,0,1


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 16 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
agrossincome      32561 non-null object
over50k           32561 non-null int64
dtypes: int64(7), object(9)
memory usage: 4.0+ MB


In [47]:
y = df.over50k
X = df.drop(['fnlwgt', 'capital-gain', 'capital-loss', 'agrossincome', 'native-country', 'sex', 'marital-status'], axis=1)
X.head(3)

,age,workclass,education,education-num,occupation,relationship,race,hours-per-week,over50k,IsMale,ISMarried
0,39,State-gov,Bachelors,13,Adm-clerical,Not-in-family,White,40,0,1,1
1,50,Self-emp-not-inc,Bachelors,13,Exec-managerial,Husband,White,13,0,1,1
2,38,Private,HS-grad,9,Handlers-cleaners,Not-in-family,White,40,0,1,1


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [42]:
model = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)

ValueError: could not convert string to float: ' White'

In [29]:
cv = KFold(n_splits=10)